In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.preprocessing import LabelEncoder

from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced

In [2]:
df = pd.read_csv('Resources/final_nba_player_dataset.csv')

players = df[['season','player','team','abbreviation']]

df = df.drop(['Unnamed: 0','season','player','team','abbreviation','league','season_id','player_id','games_started'],axis=1)
df.head()

,one_team_during_season,left_due_to_trade_during_season,joined_due_to_trade_during_season,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,...,defensive_rebounds,total_rebounds,assists,steals,blocks,turnovers,personal_fouls,points,strength_of_schedule,playoffs
0,0,1,0,14.9,0.537,0.382,0.358,5.3,18.5,11.7,...,127.0,166.0,105.0,16.0,20.0,67.0,49.0,364.0,0.29,False
1,0,0,1,14.1,0.564,0.311,0.214,6.7,13.9,10.3,...,80.0,118.0,56.0,17.0,14.0,30.0,40.0,254.0,-0.11,False
2,1,0,0,9.2,0.503,0.417,0.190,1.4,6.8,4.1,...,74.0,89.0,123.0,46.0,13.0,66.0,94.0,475.0,-0.08,False
3,1,0,0,9.4,0.573,0.607,0.157,4.6,16.6,10.6,...,99.0,127.0,23.0,15.0,9.0,23.0,87.0,218.0,-0.14,False
4,1,0,0,13.4,0.605,0.371,0.319,2.3,17.3,9.9,...,55.0,62.0,19.0,10.0,9.0,19.0,34.0,160.0,-0.15,False


In [3]:
players.head()

,season,player,team,abbreviation
0,2021,Aaron Gordon,Orlando Magic,ORL
1,2021,Aaron Gordon,Denver Nuggets,DEN
2,2021,Aaron Holiday,Indiana Pacers,IND
3,2021,Aaron Nesmith,Boston Celtics,BOS
4,2021,Abdel Nader,Phoenix Suns,PHO


In [4]:
le = LabelEncoder()

X = df.drop('playoffs', axis=1)
X = pd.get_dummies(X)

# Create our target
y = df['playoffs'].ravel()
#y.describe()

In [5]:
X.describe()

,one_team_during_season,left_due_to_trade_during_season,joined_due_to_trade_during_season,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,...,blocks,turnovers,personal_fouls,points,strength_of_schedule,position_C,position_PF,position_PG,position_SF,position_SG
count,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000,...,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000,19023.000000
mean,0.806655,0.103664,0.095726,12.576881,0.508783,0.189556,0.313033,5.998670,13.994291,9.995847,...,23.927667,70.131367,104.755191,484.076644,0.009390,0.201651,0.210377,0.196446,0.188456,0.203070
std,0.394931,0.304832,0.294223,6.409196,0.099961,0.206750,0.237317,4.898227,6.631821,4.997567,...,35.791854,65.860241,79.549514,480.205819,0.398617,0.401243,0.407587,0.397320,0.391086,0.402295
min,0.000000,0.000000,0.000000,-90.600000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,-1.030000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1.000000,0.000000,0.000000,9.700000,0.477000,0.007000,0.190000,2.400000,9.200000,6.100000,...,3.000000,16.000000,33.000000,92.000000,-0.280000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,1.000000,0.000000,0.000000,12.800000,0.521000,0.107000,0.279000,5.000000,13.000000,9.200000,...,11.000000,52.000000,95.000000,335.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,1.000000,0.000000,0.000000,15.800000,0.558000,0.336000,0.387000,8.800000,18.200000,13.400000,...,29.000000,106.000000,164.000000,747.000000,0.310000,0.000000,0.000000,0.000000,0.000000,0.000000
max,1.000000,1.000000,1.000000,133.800000,1.500000,1.000000,6.000000,100.000000,100.000000,86.400000,...,456.000000,464.000000,386.000000,3041.000000,1.170000,1.000000,1.000000,1.000000,1.000000,1.000000


In [6]:
Counter(y)

Counter({False: 8952, True: 10071})

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.3, random_state=1)
Counter(y_train)

Counter({True: 7029, False: 6287})

In [8]:
df.head()

,one_team_during_season,left_due_to_trade_during_season,joined_due_to_trade_during_season,player_efficiency_rating,true_shooting_percentage,three_point_attempt_rate,free_throw_attempt_rate,offensive_rebound_percentage,defensive_rebound_percentage,total_rebound_percentage,...,defensive_rebounds,total_rebounds,assists,steals,blocks,turnovers,personal_fouls,points,strength_of_schedule,playoffs
0,0,1,0,14.9,0.537,0.382,0.358,5.3,18.5,11.7,...,127.0,166.0,105.0,16.0,20.0,67.0,49.0,364.0,0.29,False
1,0,0,1,14.1,0.564,0.311,0.214,6.7,13.9,10.3,...,80.0,118.0,56.0,17.0,14.0,30.0,40.0,254.0,-0.11,False
2,1,0,0,9.2,0.503,0.417,0.190,1.4,6.8,4.1,...,74.0,89.0,123.0,46.0,13.0,66.0,94.0,475.0,-0.08,False
3,1,0,0,9.4,0.573,0.607,0.157,4.6,16.6,10.6,...,99.0,127.0,23.0,15.0,9.0,23.0,87.0,218.0,-0.14,False
4,1,0,0,13.4,0.605,0.371,0.319,2.3,17.3,9.9,...,55.0,62.0,19.0,10.0,9.0,19.0,34.0,160.0,-0.15,False


# Balanced Random Forest

In [9]:
from sklearn.preprocessing import StandardScaler
#from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier

# Creating a StandardScaler instance.
scaler = StandardScaler()
# Fitting the Standard Scaler with the training data.
X_scaler = scaler.fit(X_train)

# Scaling the data.
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Create a random forest classifier.
rf_model = BalancedRandomForestClassifier(n_estimators=128, random_state=1) 

# Fitting the model
rf_model = rf_model.fit(X_train_scaled, y_train)
predictions = rf_model.predict(X_test_scaled)

In [10]:
#Apply model to whole dataset
predict=rf_model.predict(X)
df['preds'] = predict

#merge with players_df
players = pd.merge(players, df, how='left',left_index=True, right_index=True)
players.to_csv('Resources/balanced_forest_predictions.csv', index=False)

In [11]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Calculating the confusion matrix.
cm = confusion_matrix(y_test, predictions)

y_pred = rf_model.predict(X_test)
confusion_matrix(y_test, y_pred)

#print(cm)
# Create a DataFrame from the confusion matrix.
cm_df = pd.DataFrame(
    cm, index=["Actual 0", "Actual 1"], columns=["Predicted 0", "Predicted 1"])

cm_df

,Predicted 0,Predicted 1
Actual 0,2048,617
Actual 1,718,2324


In [12]:
# Calculating the accuracy score.
acc_score = accuracy_score(y_test, predictions)

print(acc_score)

0.7660767478535132


In [13]:
# Print the imbalanced classification report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

       False       0.74      0.77      0.75      2665
        True       0.79      0.76      0.78      3042

    accuracy                           0.77      5707
   macro avg       0.77      0.77      0.77      5707
weighted avg       0.77      0.77      0.77      5707



In [14]:
# List the features sorted in descending order by feature importance
importances = rf_model.feature_importances_
importances
# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.137726550925583, 'strength_of_schedule'),
 (0.0557385251235588, 'win_shares_per_48_min'),
 (0.051707641448394695, 'defensive_win_shares'),
 (0.038826384901741735, 'defensive_box_plus/minus'),
 (0.028374755592382943, 'win_shares'),
 (0.028332231507998483, 'player_efficiency_rating'),
 (0.02486194960855127, 'usage_percentage'),
 (0.024267074808351004, 'defensive_rebounds'),
 (0.024261172219314772, 'defensive_rebound_percentage'),
 (0.024172217954890166, 'games'),
 (0.023098144430471076, 'true_shooting_percentage'),
 (0.02231075672967461, 'total_rebounds'),
 (0.021786502698308195, 'minutes_played'),
 (0.020999933648884826, 'box_plus/minus'),
 (0.020681105307061545, 'offensive_box_plus/minus'),
 (0.019419978487142598, 'free_throw_attempt_rate'),
 (0.019369847600005985, 'age'),
 (0.019261231262192847, 'total_rebound_percentage'),
 (0.01853430527236344, 'personal_fouls'),
 (0.01839422578804022, 'assist_percentage'),
 (0.018271253982843485, 'steals'),
 (0.01809777708123822, 'offensive_reb

# Easy Ensemble AdaBoost Classifier

In [15]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier

# Create a easy ensemble classifier
ada_model = EasyEnsembleClassifier(n_estimators=128, random_state=1) 

# Fitting the model
ada_model = ada_model.fit(X_train_scaled, y_train)
predictions = ada_model.predict(X_test_scaled)

In [16]:
# Calculated the balanced accuracy score
y_pred2 = ada_model.predict(X_test)
confusion_matrix(y_test, y_pred)

print(cm)

[[2048  617]
 [ 718 2324]]


In [17]:
# Display the confusion matrix
acc_score = accuracy_score(y_test, predictions)

print(acc_score)

0.765375854214123


In [18]:
# Print the imbalanced classification report

print(classification_report_imbalanced(y_test, predictions))

                   pre       rec       spe        f1       geo       iba       sup

      False       0.74      0.78      0.76      0.76      0.77      0.59      2665
       True       0.79      0.76      0.78      0.77      0.77      0.59      3042

avg / total       0.77      0.77      0.77      0.77      0.77      0.59      5707

